In [10]:
import numpy as np
import pandas as pd
import skimage as img
import matplotlib.pyplot as plt
import math
import cv2

from skimage.filters import sobel
from scipy import ndimage as ndi
from skimage.color import label2rgb
from skimage import measure
import matplotlib.patches as mpatches
from IPython.display import clear_output

In [3]:
# complete function
def colony_count(plate):
    clear_output(wait = True)
    markers = np.zeros_like(plate)
    markers[plate < 100] = 1
    markers[plate > 220] = 2

    # use Sobel image segmentation and fill holes
    fill = ndi.binary_fill_holes(sobel(plate > 220))
    
    # clean the plate by removing segments that are too small
    label_objects, nb_labels = ndi.label(fill)
    sizes = np.bincount(label_objects.ravel())
    mask_sizes = (sizes > 300) & (sizes < 800)

    # label the colonies
    mask_sizes[0] = 0
    plate_cleaned = mask_sizes[label_objects]
    labeled_plate, _ = ndi.label(plate_cleaned)
    image_label_overlay = label2rgb(labeled_plate, image=plate, bg_label=0)
    
    # only segment labels that are circular in shape
    props = measure.regionprops_table(labeled_plate, properties=('label', 'area', 'perimeter'))
    table = pd.DataFrame(props)
    # calculate the thinness ratio (1 equals a circle)
    table = table.assign(thinness_ratio = 4 * math.pi * table.area / (table.perimeter * table.perimeter))
    
    # show only segments with a thinness_ratio of 0.7 or higher
    print("This plate contains", table[table.thinness_ratio >= 0.7].shape[0], "colonies.")

    # plot the final picture
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image_label_overlay)

    for region in measure.regionprops(labeled_plate):
        # take regions with large enough areas
        if 4 * math.pi * region.area / (region.perimeter * region.perimeter) >= 0.70:
            # draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='red', linewidth=1, alpha = .7)
            ax.add_patch(rect)

    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

Run the chunk below. To quit press 'ESC', to count the colonies on a plate press 'SPACE'

This chunk beneath starts the webcam. When spacebar is pressed, it will take the current frame, converts it to grayscale and counts the colonies on the plate.

In [ ]:
# define a video capture object
vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

vid.set(28, 50)

while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
  
    # Display the resulting frame
    cv2.imshow('frame', frame)
          
    k = cv2.waitKey(1)
    if k%256 == 27: # press ESC to quit
        break
    elif k%256 == 32: # press SPACE to count colonies
        # SPACE pressed
        plate = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        colony_count(plate)

  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()